In [1]:
import sys
import numpy as np
import matplotlib.pyplot as plt
import scipy.constants as cts
import pylcp
import time
import random
from tqdm import tqdm
from scipy.interpolate import interp1d,interp2d,RegularGridInterpolator
from scipy import stats
from scipy import integrate
from celluloid import Camera
from bayes_opt import BayesianOptimization
import json
import seaborn as sns
import White_class
from scipy import special
import pymongo
import Expected_model_simulation
from pymongo import MongoClient
from scipy.optimize import root_scalar


c:\Users\qmopl\anaconda3\envs\LCPtest\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
#Main variables

main_detune = 17
sideband_detune = 120
white_detune = 10
laser_on = 200000
laser_off = 400015
MOT_power = 50
v0_start=1
v0_step=1
v0_end=25
t0_start=0
t0_step=1
t0_end=1000000


Gamma = 22 # Hz to kHz, Decay rate
wavelength = 359.3e-9 # m to mm
k = 1/wavelength*2*np.pi #x_0
x0 = 1/k
t0 = 1/Gamma*1/(2*np.pi*1e6)
v0 = x0/t0
m0 = cts.hbar*t0/x0**2
a0 = x0/t0**2
F0 = cts.hbar/(x0*t0)
mass = 43*cts.value('atomic mass constant')/m0 # m_0
waist = 0.012/x0
v_max = 25.
z_max = 384.855e-3/x0
z_start = 384.855e-3/x0
omega = 2*np.pi*(cts.c/wavelength) #Transition frequency
Isat = np.pi*cts.h*cts.c*Gamma*2*np.pi*1e6/3*1/(wavelength)**3
t_eval = np.arange(t0_start,t0_end,t0_step)

# The detunings used in the PRAs:
intensities = 2.*MOT_power*1e-3/(np.pi*0.012**2)/Isat

#Define the hamiltonian
H0_X, Bq_X, U_X, Xbasis = pylcp.hamiltonians.XFmolecules.Xstate(N=1,I=0.5,B=15496.8125/Gamma,
gamma = 50.697/Gamma,b=154.7/Gamma, c=178.5/Gamma,gI=5.585,gS=2.0023193043622,
    muB = cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma,
    muN=cts.m_e/cts.m_p*cts.value('Bohr magneton in Hz/T')*1e-4*1e-6/Gamma,return_basis=True
    )

# b : SI coupling(isotropic), c : Iz Sz coupling(anisotropic), cc : I N coupling, gamma : S N coupling

E_X = np.unique(np.diag(H0_X))

H0_A, Bq_A, Abasis = pylcp.hamiltonians.XFmolecules.Astate(J=0.5,I=0.5,
    P=+1,B=15788.2/Gamma,D=0.,H=0.,a=109./Gamma,b=-299.2/Gamma,c=274.2/Gamma,q=0., p=15./Gamma,
    muB=cts.value('Bohr magneton in Hz/T')/1e6*1e-4/Gamma,
    muN=cts.m_e/cts.m_p*cts.value('Bohr magneton in Hz/T')*1e-4*1e-6/Gamma,
    gl=53/(2*15788.2),glprime=15/(2*15788.2),greprime=0.,return_basis=True
    )
# gJ : Lande g-factor, p : parity(e parity)


E_A = np.unique(np.diag(H0_A))

dijq = pylcp.hamiltonians.XFmolecules.dipoleXandAstates(
    Xbasis, Abasis, UX=U_X
    )

hamiltonian = pylcp.hamiltonian(H0_X, H0_A, Bq_X, Bq_A, dijq,mass = mass)

In [2]:
connection = MongoClient("localhost:27017")
db = connection.db.Expected_model
max_parameters = db.find(projection={"_id":0},limit=5).sort("target",pymongo.DESCENDING)

In [4]:
max_parameters[0]

{'target': 0.013434975559152768,
 'params': {'beta_1': 2.935124145664613,
  'beta_2': 10.312814186442964,
  'det_1': 115.49496654453418,
  'det_2': 15.388812962322799,
  'laseroff': 411581.2063215908,
  'laseron': 235170.32507489598,
  'main_det': 19.67630442684696},
 'datetime': {'datetime': '2022-06-26 17:54:32',
  'elapsed': 88663.352371,
  'delta': 143.803469}}

In [7]:
start = time.time()

print(Expected_model_simulation.Expected_simulation(**max_parameters[0]["params"]))

print(time.time()-start)

0.011812525646444817
174.9402084350586


In [3]:
pbounds = {'main_det':(10,20),'det_1':(70,170),'det_2':(5,30),'beta_1':(0,5),'beta_2':(0,15),'laseron':(138240,300000),'laseroff':(300001,600000)}

In [6]:
optimizer = BayesianOptimization(
    f = Expected_model_simulation.Expected_simulation,
    pbounds=pbounds,
    verbose=1,
    random_state=1,
)

In [5]:
from bayes_opt.util import load_logs

# New optimizer is loaded with previously seen points
load_logs(optimizer, logs=["./logs_Exp_multi.json"])

In [7]:
start = time.time()
optimizer.maximize(
    init_points=1,
    n_iter=0,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
140.133859872818


In [8]:
from bayes_opt.logger import JSONLogger
from bayes_opt.event import Events

logger = JSONLogger(path='./logs_Exp_multi.json')
optimizer.subscribe(Events.OPTIMIZATION_STEP,logger)

In [9]:
optimizer.max

{'target': 0.0,
 'params': {'beta_1': 2.08511002351287,
  'beta_2': 10.804867401632372,
  'det_1': 70.01143748173449,
  'det_2': 12.558314315795993,
  'laseroff': 344027.62048924307,
  'laseron': 153176.69108980073,
  'main_det': 11.86260211377671}}

In [10]:
start = time.time()
optimizer.maximize(
    init_points=5,
    n_iter=10,)
print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  3        |  0.001336 |  0.1369   |  10.06    |  111.7    |  18.97    |  3.421e+0 |  1.703e+0 |  18.01    |
|  11       |  0.01069  |  0.9655   |  5.693    |  75.41    |  7.451    |  3.43e+05 |  1.691e+0 |  19.41    |
2056.2213096618652


In [12]:
start = time.time()
optimizer.maximize(
    init_points=5,
    n_iter=25,)
print(time.time()-start)


|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  62       |  0.01343  |  2.935    |  10.31    |  115.5    |  15.39    |  4.116e+0 |  2.352e+0 |  19.68    |
3708.367482662201


In [11]:
optimizer.maximize(n_iter=1,init_points = 0)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  47       |  0.005689 |  2.149    |  0.3916   |  103.7    |  19.79    |  4.88e+05 |  2.369e+0 |  19.61    |


In [14]:
start = time.time()
optimizer.maximize(n_iter=5,init_points = 45)

print(time.time()-start)

|   iter    |  target   |  beta_1   |  beta_2   |   det_1   |   det_2   | laseroff  |  laseron  | main_det  |
-------------------------------------------------------------------------------------------------------------
|  98       |  0.01181  |  2.705    |  14.39    |  150.4    |  5.808    |  5.128e+0 |  2.135e+0 |  19.48    |
7503.4828906059265


In [15]:
optimizer.max

{'target': 0.011812525646444817,
 'params': {'beta_1': 2.704759024810943,
  'beta_2': 14.391514815926286,
  'det_1': 150.39608914986468,
  'det_2': 5.80807666558935,
  'laseroff': 512816.4659821373,
  'laseron': 213458.63965771275,
  'main_det': 19.475489413950996}}

In [16]:
len(optimizer.res)

146

In [36]:
rest = db.find(projection={"_id" : 0}).sort("target",-1).limit(3)

In [37]:
for res in rest:
    print(res['params'])


{'beta_1': 2.935124145664613, 'beta_2': 10.312814186442964, 'det_1': 115.49496654453418, 'det_2': 15.388812962322799, 'laseroff': 411581.2063215908, 'laseron': 235170.32507489598, 'main_det': 19.67630442684696}
{'beta_1': 2.704759024810943, 'beta_2': 14.391514815926286, 'det_1': 150.39608914986468, 'det_2': 5.80807666558935, 'laseroff': 512816.4659821373, 'laseron': 213458.63965771275, 'main_det': 19.475489413950996}
{'beta_1': 2.704759024810943, 'beta_2': 14.391514815926286, 'det_1': 150.39608914986468, 'det_2': 5.80807666558935, 'laseroff': 512816.4659821373, 'laseron': 213458.63965771275, 'main_det': 19.475489413950996}
